In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split

In [2]:
import pickle
import pandas as pd
import numpy
import re
import os
import numpy as np
import gensim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from gensim.models import Doc2Vec

In [3]:
df_mn = pd.read_csv('./sutta_csv/cleaned/df_mn.csv')
df_mn2 = df_mn.copy()

## Lowercasing

In [4]:
df_mn2['text_full'] = df_mn2['text_full'].str.lower()

In [5]:
df_mn2.head(1)

title  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence   

                                        title_url   ref  \
0  https://www.dhammatalks.org/suttas/MN/MN1.html  MN 1   

                                                                                                                                                                                                                                                                                                    summary  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence  —  The Majjhima Nikāya opens with one of the few suttas where his listeners did NOT delight in his words. In it, the Buddha dismisses the tendency—common both in his time and in ours—to posit a metaphysical principle from which the universe emanates.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Removing punctuation

In [6]:
import string

In [7]:
#df_mn2['text_full'] = df_mn2['text_full'].translate(str.maketrans('', '', string.punctuation))

In [8]:
df_mn2['text_full'] = df_mn2['text_full'].str.replace('.', ' ')
df_mn2['text_full'] = df_mn2['text_full'].str.replace('-', ' ')
df_mn2['text_full'] = df_mn2['text_full'].str.replace('?', ' ')
df_mn2['text_full'] = df_mn2['text_full'].str.replace('(', ' ')
df_mn2['text_full'] = df_mn2['text_full'].str.replace(')', ' ')
df_mn2['text_full'] = df_mn2['text_full'].str.replace('  ', ' ')

In [9]:
df_mn2['text_full'] = df_mn2['text_full'].str.replace('[^\w\s]', "")

In [10]:
# Good insight into how to apply function to dataframe

#https://www.pythondaddy.com/python/how-to-remove-punctuation-from-a-dataframe-in-pandas-and-python/
# def remove_punctuation(text):
#     try:
#         text = text.str.replace('[^\w\s]','')
#     except:
#         pass
#     return text

# df_mn2['text_full'].apply(remove_punctuation)

In [11]:
df_mn2.head(1)

title  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence   

                                        title_url   ref  \
0  https://www.dhammatalks.org/suttas/MN/MN1.html  MN 1   

                                                                                                                                                                                                                                                                                                    summary  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence  —  The Majjhima Nikāya opens with one of the few suttas where his listeners did NOT delight in his words. In it, the Buddha dismisses the tendency—common both in his time and in ours—to posit a metaphysical principle from which the universe emanates.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Tokenzing (code for Splitting at \n)

In [12]:
df_mn2['text_token'] = [simple_preprocess(line, deacc=True) for line in df_mn2['text_full']] 

In [13]:
df_mn2.head(1)

title  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence   

                                        title_url   ref  \
0  https://www.dhammatalks.org/suttas/MN/MN1.html  MN 1   

                                                                                                                                                                                                                                                                                                    summary  \
0  MN 1  Mūlapariyāya Sutta | The Root Sequence  —  The Majjhima Nikāya opens with one of the few suttas where his listeners did NOT delight in his words. In it, the Buddha dismisses the tendency—common both in his time and in ours—to posit a metaphysical principle from which the universe emanates.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [14]:
len(df_mn2['text_token'][0])

1742

In [15]:
#df_mn2['text_full'] = df_mn2['text_full'].str.split("\n")

## Removing Stop Words

In [16]:
# https://www.analyticsvidhya.com/blog/2018/10/stepwise-guide-topic-modeling-latent-semantic-analysis/
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# remove stop-words
df_mn2['text_no_stop'] = df_mn2['text_token'].apply(lambda x: [item for item in x if item not in stop_words])

# Modeling 

## Topic Analysis with TfidF and Latent Semantic Analysis

### First using tfidf

In [17]:
df_mn2['text_full'].head(1)

0    \n     i have heard that on one occasion the blessed one was staying near ukkaṭṭhā in the shade of a royal sal tree in the very blessed forest there he addressed the monks monks\n     yes lord the monks responded to him \n     the blessed one said monks i will teach you the sequence of the root of all phenomena or the root sequence of all phenomena listen  pay close attention i will speak \n     as you say lord they responded to him \n     the blessed one said there is the case monks where an uninstructed run of the mill personwho has no regard for noble ones is not well versed or disciplined in their dhamma who has no regard for people of integrity is not well versed or disciplined in their dhammaperceives earth as earth perceiving earth as earth he supposes things about earth he supposes things in earth he supposes things coming out of earth he supposes earth as mine he delights in earth why is that because he has not comprehended it i tell you \n     he perceives water as water

In [18]:
# https://www.analyticsvidhya.com/blog/2018/10/stepwise-guide-topic-modeling-latent-semantic-analysis/
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 3000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(df_mn2['text_full'])

X.shape # check shape of the document-term matrix

(103, 3000)

#### Using truncastedSVD to create document-term matrix and then break into multiple

In [19]:
# https://www.analyticsvidhya.com/blog/2018/10/stepwise-guide-topic-modeling-latent-semantic-analysis/

from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=50, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

#These are our topics
len(svd_model.components_)

50

In [20]:
## Print out the relevant topics
# https://www.analyticsvidhya.com/blog/2018/10/stepwise-guide-topic-modeling-latent-semantic-analysis/

terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")

Topic 0: 
pleasure
 
self
 
feeling
 
view
 
consciousness
 
gotama
 
qualities
 
Topic 1: 
inconstant
 
consciousness
 
self
 
contact
 
feeling
 
property
 
clinging
 
Topic 2: 
introduction
 
discourse
 
papañca
 
simile
 
clinging
 
leading
 
similes
 
Topic 3: 
inconstant
 
gotama
 
master
 
view
 
self
 
stressful
 
constant
 
Topic 4: 
heartwood
 
property
 
introduction
 
inconstant
 
consummation
 
gotama
 
breathe
 
Topic 5: 
heartwood
 
consummation
 
fame
 
offerings
 
gain
 
knowledge
 
vision
 
Topic 6: 
heartwood
 
consummation
 
view
 
breathe
 
property
 
resolve
 
knowledge
 
Topic 7: 
integrity
 
person
 
view
 
dimension
 
perception
 
quality
 
wrong
 
Topic 8: 
sensuality
 
drawback
 
inconstant
 
reason
 
source
 
stress
 
action
 
Topic 9: 
inconstant
 
stressful
 
subject
 
change
 
constant
 
ven
 
venerable
 
Topic 10: 
integrity
 
inconstant
 
person
 
quality
 
sensuality
 
stressful
 
breathe
 
Topic 11: 
pleasure
 
torturing
 
torments
 
devoted
 
conduct

### Visualizing 

## Model 2: LDA Model

In [21]:
# https://towardsdatascience.com/topic-modeling-with-latent-dirichlet-allocation-e7ff75290f8

In [22]:
# https://highdemandskills.com/topic-modeling-lda/

## Preprocessing for Doc2Vec Specifically

In [23]:
## https://medium.com/@ermolushka/text-clusterization-using-python-and-doc2vec-8c499668fa61
#Converting into form doc2vec expects

LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
mn_text = []
j=0

for em in df_mn2['text_full'].values:
    mn_text.append(LabeledSentence1(em,[j]))
    j+=1
print('Number of texts processed: ', j)

Number of texts processed:  103


In [24]:
#saving processed text in a new column
df_mn2['processed'] = mn_text

## doc2vec

In [25]:
## Methodology from the lab

# model = Doc2Vec(df_mn2['processed'])
# model.train(df_mn2['processed'], total_examples=model.corpus_count, epochs = model.epochs)

# ## https://medium.com/@ermolushka/text-clusterization-using-python-and-doc2vec-8c499668fa61

# kmeans_model = KMeans(n_clusters=4, init='k-means++', max_iter=100) 
# X = kmeans_model.fit(model)
# labels=kmeans_model.labels_.tolist()

# l = kmeans_model.fit_predict(model)
# pca = PCA(n_components=2).fit(model)
# datapoint = pca.transform(model)

## Model 3: LDA

In [26]:
import gensim
import gensim.corpora as corpora
from gensim import models
import matplotlib.pyplot as plt
from pprint import pprint
from wordcloud import WordCloud
from mpl_toolkits import mplot3d
import gensim.corpora as corpora

In [27]:
##https://highdemandskills.com/topic-modeling-lda/

In [28]:
#### Step 1: Form dictionary by mapping corpus to words id and convert using bag of words

In [29]:
## Removing \n
df_mn2['text_full'] = df_mn2['text_full'].str.replace('\n', ' ')
df_mn2['text_full'] = df_mn2['text_full'].str.replace('      ', ' ')

In [33]:
## Creating a list of all the text elements
text_mn = df_mn2['text_token'].to_list()

In [34]:
text_mn

[['have',
  'heard',
  'that',
  'on',
  'one',
  'occasion',
  'the',
  'blessed',
  'one',
  'was',
  'staying',
  'near',
  'ukkattha',
  'in',
  'the',
  'shade',
  'of',
  'royal',
  'sal',
  'tree',
  'in',
  'the',
  'very',
  'blessed',
  'forest',
  'there',
  'he',
  'addressed',
  'the',
  'monks',
  'monks',
  'yes',
  'lord',
  'the',
  'monks',
  'responded',
  'to',
  'him',
  'the',
  'blessed',
  'one',
  'said',
  'monks',
  'will',
  'teach',
  'you',
  'the',
  'sequence',
  'of',
  'the',
  'root',
  'of',
  'all',
  'phenomena',
  'or',
  'the',
  'root',
  'sequence',
  'of',
  'all',
  'phenomena',
  'listen',
  'pay',
  'close',
  'attention',
  'will',
  'speak',
  'as',
  'you',
  'say',
  'lord',
  'they',
  'responded',
  'to',
  'him',
  'the',
  'blessed',
  'one',
  'said',
  'there',
  'is',
  'the',
  'case',
  'monks',
  'where',
  'an',
  'uninstructed',
  'run',
  'of',
  'the',
  'mill',
  'personwho',
  'has',
  'no',
  'regard',
  'for',
  'noble

In [35]:
##Converting all into dictionary of ID
ID2word = corpora.Dictionary(text_mn)

In [36]:
ID2word

In [37]:
NUM_topics = 5 # Set number of topics

# Train LDA model on the training corpus
lda_model = gensim.models.LdaMulticore(corpus=text_mn, num_topics=NUM_topics, id2word=ID2word, passes=100)

KeyboardInterrupt: 